<a href="https://colab.research.google.com/github/migsfranco/awana-skills-program/blob/main/Sf50Awana_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JOB DESCRIPTIONS MATCHING WITH CANDIDATE'S CV/RESUME

---



In [2]:
# Leer un secreto que se llama OpenAIAPIKEY guardado en el acceso de secretos de Google Colab

import os

# Access the secret
try:
  api_key = os.environ['OpenAIAPIKEY']
  print("API Key loaded successfully from secrets.")
except KeyError:
  print("Error: OpenAIAPIKEY not found in secrets.")
  api_key = None # or handle the error as appropriate

# Now you can use the api_key
if api_key:
  print(f"API Key (first 5 chars): {api_key[:5]}...") # Example usage, don't print the entire key
else:
  print("API key not available.  Cannot continue.")


Error: OpenAIAPIKEY not found in secrets.
API key not available.  Cannot continue.


In [ ]:
# 1. The first instruction provides useful tools for controlling output in interactive environments such as Jupyter Notebooks or Google Colab
#and clear_output → clear cell output before printing a new iteration.
from IPython.display import clear_output

In [ ]:
# 2. Install the dependencies for this project.
!pip install pandas
!pip install requests
!pip install numpy
!pip install nltk
!pip install gensim
!pip install PyPDF2


In [ ]:
# 3. Let's import the libraries

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import requests
import PyPDF2
import re
import plotly.graph_objects as go
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# 4. Import the dataset from Kaggle
data_2009_to_2015 = pd.read_csv('/content/Datos/data job posts_2004_a_2015.csv')

In [ ]:
#4.1 Data that is contained in this variable
data_2009_to_2015

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18996,Technolinguistics NGO\r\n\r\n\r\nTITLE: Senio...,"Dec 28, 2015",Senior Creative UX/ UI Designer,Technolinguistics NGO,NaN,Full-time,NaN,NaN,NaN,Long-term,...,Competitive,"To apply for this position, please send your\r...",29 December 2015,28 January 2016,NaN,As a company Technolinguistics has a mandate t...,NaN,2015,12,False
18997,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Dec 30, 2015",Category Development Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,Full-time,All interested professionals.,NaN,ASAP,Long-term with a probation period of 3 months.,...,NaN,All interested candidates are kindly requested...,30 December 2015,20 January 2016,NaN,NaN,NaN,2015,12,False
18998,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Dec 30, 2015",Operational Marketing Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,Full-time,All interested professionals.,NaN,ASAP,Long-term with a probation period of 3 months.,...,NaN,All interested candidates are kindly requested...,30 December 2015,20 January 2016,NaN,NaN,NaN,2015,12,False
18999,San Lazzaro LLC\r\n\r\n\r\nTITLE: Head of O...,"Dec 30, 2015",Head of Online Sales Department,San Lazzaro LLC,NaN,NaN,NaN,NaN,NaN,Long-term,...,Highly competitive,Interested candidates can send their CVs to:\r...,30 December 2015,29 January 2016,NaN,San Lazzaro LLC works with several internation...,NaN,2015,12,False


In [ ]:
# 4.1.1 Show the column names, beacuse with head does not show all the data
data_2009_to_2015.columns

Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')

In [ ]:
# 5. Let's gonna keep some columns just to train our model
data_2009_to_2015 = data_2009_to_2015[['jobpost', 'Title', 'Company', 'Location', 'JobRequirment', 'Salary']]
data_2009_to_2015

,jobpost,Title,Company,Location,JobRequirment,Salary
0,AMERIA Investment Consulting Company\r\nJOB TI...,Chief Financial Officer,AMERIA Investment Consulting Company,"Yerevan, Armenia",- Supervises financial management and administ...,NaN
1,International Research & Exchanges Board (IREX...,Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),"IREX Armenia Main Office; Yerevan, Armenia \r\...",NaN,NaN
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,Country Coordinator,Caucasus Environmental NGO Network (CENN),"Yerevan, Armenia",- Working with the Country Director to provide...,NaN
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,BCC Specialist,Manoff Group,"Manila, Philippines",- Identify gaps in knowledge and overseeing in...,NaN
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,Software Developer,Yerevan Brandy Company,"Yerevan, Armenia",- Rendering technical assistance to Database M...,NaN
...,...,...,...,...,...,...
18996,Technolinguistics NGO\r\n\r\n\r\nTITLE: Senio...,Senior Creative UX/ UI Designer,Technolinguistics NGO,"Yerevan, Armenia",- Work closely with product and business teams...,Competitive
18997,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",Category Development Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Yerevan, Armenia",- Establish and manage Category Management dev...,NaN
18998,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",Operational Marketing Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Yerevan, Armenia","- Develop, establish and maintain marketing st...",NaN
18999,San Lazzaro LLC\r\n\r\n\r\nTITLE: Head of O...,Head of Online Sales Department,San Lazzaro LLC,"Yerevan, Armenia",- Handle the project activites of the online s...,Highly competitive


In [ ]:
# Create a new column called 'data' and merge the values of the other columns into it
data_2009_to_2015['data'] = data_2009_to_2015[['jobpost', 'Title', 'Company', 'Location', 'JobRequirment', 'Salary']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Drop the individual columns if you no longer need them
data_2009_to_2015.drop(columns=['jobpost', 'Title', 'Company', 'Location', 'JobRequirment', 'Salary'], inplace=True)

# Preview the updated dataframe
print(data_2009_to_2015.head())


                                                data
0  AMERIA Investment Consulting Company\r\nJOB TI...
1  International Research & Exchanges Board (IREX...
2  Caucasus Environmental NGO Network (CENN)\r\nJ...
3  Manoff Group\r\nJOB TITLE:  BCC Specialist\r\n...
4  Yerevan Brandy Company\r\nJOB TITLE:  Software...


<ipython-input-13-e86f76c22264>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2009_to_2015['data'] = data_2009_to_2015[['jobpost', 'Title', 'Company', 'Location', 'JobRequirment', 'Salary']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
<ipython-input-13-e86f76c22264>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2009_to_2015.drop(columns=['jobpost', 'Title', 'Company', 'Location', 'JobRequirment', 'Salary'], inplace=True)


In [ ]:
#Let's show the information again with the changes
data_2009_to_2015

,data
0,AMERIA Investment Consulting Company\r\nJOB TI...
1,International Research & Exchanges Board (IREX...
2,Caucasus Environmental NGO Network (CENN)\r\nJ...
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...
4,Yerevan Brandy Company\r\nJOB TITLE: Software...
...,...
18996,Technolinguistics NGO\r\n\r\n\r\nTITLE: Senio...
18997,"""Coca-Cola Hellenic Bottling Company Armenia"" ..."
18998,"""Coca-Cola Hellenic Bottling Company Armenia"" ..."
18999,San Lazzaro LLC\r\n\r\n\r\nTITLE: Head of O...


In [ ]:
# Data Tokenization
import nltk                # this library helps us work with words and phrases.
nltk.download('punkt_tab') # Divides the sentences into words. punk_tab is like a list that helps machine to know where the words begin and end
data = list(data_2009_to_2015['data']) # Here, we are taking a data column of our dataset and gathering the phrases and putting them in a box to later use them.
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Model initialization. Doc2Vec is a model that represents each document as a vector.
model = Doc2Vec(vector_size = 50,
min_count = 5, # Focus the attention on the words that appear at least , in this ce it is five words.
epochs = 3, # Here, we tell the machine to study for a specific time.
alpha = 0.001)  # This is a number that mentions to a machine on how fast it should learn. This number indicates a slow pace of learning.

In [ ]:
# Creation of a Vocabulary building  -> Here, we are telling to our machine to learn all the words
# from our phrases just like a word dictionary.
model.build_vocab(tagged_data)
# Get the vocabulary keys -> Here, we are taking out the words that were learned by the machine
#and save it into a list named keys
keys = model.wv.key_to_index.keys()
# Print the length of the vocabulary keys (Counting words)
print(len(keys))

19029


In [ ]:
data_2009_to_2015

,data
0,AMERIA Investment Consulting Company\r\nJOB TI...
1,International Research & Exchanges Board (IREX...
2,Caucasus Environmental NGO Network (CENN)\r\nJ...
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...
4,Yerevan Brandy Company\r\nJOB TITLE: Software...
...,...
18996,Technolinguistics NGO\r\n\r\n\r\nTITLE: Senio...
18997,"""Coca-Cola Hellenic Bottling Company Armenia"" ..."
18998,"""Coca-Cola Hellenic Bottling Company Armenia"" ..."
18999,San Lazzaro LLC\r\n\r\n\r\nTITLE: Head of O...


In [ ]:
# Train and save the model on tagged data
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)

model.save('cv_job_maching.model')
print("Model saved")

Training epoch 1/3


Training epoch 2/3


Training epoch 3/3
Model saved


In [ ]:
#Inputs of CV

pdf = PyPDF2.PdfReader('/content/CV/Vicky-Valverde-CV-EN.pdf') # PyPDF is used to read the content of the pdf inserted
resume = ""                       # Here, I create a void box called resume
for i in range(len(pdf.pages)):   #Go through each page of the curriculum.
    pageObj = pdf.pages[i]        # Open a page and extract the text. Here we have a specific page.
    resume += pageObj.extract_text() # Here, we add more text to what I have in the box.

In [ ]:
#Here, we visualize the resume of the candidate which was uploaded previously
resume

'V\nICKY\n \nV\nALVERDE\nPROFILE\nI am a Software Developer with experience in frontend \ndevelopment using React, JavaScript, TypeScript, and Angular. I \nhave also worked on projects related to mobile development \nusing React Native, Android, and XCode and solutions with \nNode and Java.\nI like to solve problems by using my technical expertise to find \nthe simplest yet effective solution.\nI believe that technology is changing our lifestyle, and it is time \nto be part of this change.\nvickyvalverdeg@gmail.com\n+593 991367161\nhttps://www.linkedin.com/in/vickyvalverdeg\n \nLANGUAGES\nWORK EXPERIENCE\n● \nSpanish (native \nlanguage)\n● \nIEnglish: Professional \nwork competence \n(ECCE B2 Certificate).\nThoughtworks\nEcuador, Quito\n2018 - 2024\nI have worked on various projects related to frontend, backend, \nand mobile solutions. The last few years I’ve been working with \nReact, Angular, Typescript, Javascript, and Node.js.\nImage Tech – Web & Multimedia Solutions\nEcuador, Guay

In [ ]:
#Here, we put an example of a job description, focusing on Name of the position, key responsibilities, must have, nice to have
#demosntrated experience

job_desc = '''"PositionID":"FMCSA.MC-I-2025-0002",
        "PositionTitle":"IT Specialist (Systems Engineer) - DIRECT HIRE",
        "QualificationSummary":"To meet the minimum qualifications for this position, you must meet the specialized experience requirements. To qualify for the GS-14, you must have at least one year of experience equal or equivalent to the GS-13, it must include: Experience in assisting, supporting in planning, designing, developing, and integrating network systems consistent with cloud (AWS, Azure, etc.) and\/or planned network infrastructures. Experience analyzing and defining network requirements, network workload, monitoring network capacity and performance, diagnosing, and resolving network problems, for cloud (AWS, Azure and etc.). Experience in assisting and supporting in the development, implementation, and ensuring compliance with plans, policies, standards, infrastructures, and architectures that establish the framework for the management of IT programs. Experience in leading the initiative of implementing robotic process automation at the agency level working with business areas, collecting their business cases and deliver automation using standard process automation tools. You cannot qualify on education alone at the GS-14; you must also have experience. KNOWLEDGE, SKILLS AND ABILITIES (KSAs): Your qualifications will be evaluated on the basis of your level of knowledge, skills, abilities and\/or competencies in the following areas: Systems Engineering - Knowledge of the practice of integrating multiple disciplines into a team as part of a structured development process throughout a system's life cycle. Engineering and Technology - Knowledge of engineering concepts, principles, and practices, and of equipment, tools, mechanical devices, and their uses to produce motion, light, power, technology, and other applications. IT Program Management - Knowledge of the principles, methods, and tools for the coordinated management of an IT program to include providing oversight of multiple IT projects, integrating dependent schedules and deliverables, and related activities (for example, benefits management, life cycle management, program governance). Technology Application - Uses machines, tools, instruments, or equipment effectively; uses computers and computer applications to analyze and communicate information in the appropriate format. Electronics Engineering - Knowledge of the concepts, principles, theories, and methods related to the design, analysis, test, fabrication, or verification of analog or digital electronic systems. In addition to meeting the above described specialized experience requirements, your IT experience will be evaluated and be expected to include described experience in the following competencies: Attention to Detail: Work is thorough and conscientious about attending to detail. Customer Service: Working with clients and customers (that is, any individuals who use or receive the services or products that your work unit produces, including, individuals who work in the agency, other agencies, or organizations outside the Government) to assess their needs, providing information or assistance, assigning staff to resolve their problems, or satisfying their expectations; knowing about available products and services; committed to providing quality products and services. Oral Communication: Expressing information (for example, ideas or facts) to individuals or groups effectively, taking into account the audience and nature of the information (for example, technical, sensitive, controversial); making clear and convincing oral presentations; listening to others, attending to nonverbal cues, and responding appropriately. Problem Solving: Identifying problems; determining accuracy and relevance of information; using sound judgment to generate and evaluate alternatives and making recommendations based on technical requirements. Experience refers to paid and unpaid experience, including volunteer work done through National Service programs (e.g., Peace Corps, AmeriCorps) and other organizations (e.g., professional, philanthropic, religious, spiritual, community, student, social). Volunteer work helps build critical competencies, knowledge, and skills and can provide valuable training and experience that translates directly to paid employment. You will receive credit for all qualifying experience, including volunteer experience. For additional information about applying to Federal positions, please click on the following link: https:\/\/www.opm.gov\/frequently-asked-questions\/employment-faq\/federal-hiring\/how-do-i-apply-for-federal-jobs\/ All applicants must meet all qualification requirements by the closing date of this announcement.",
        "JobSummary":"The Information Technology Specialist provides technical leadership and hands-on design support for new systems, enhancing\/refactoring existing applications, and executing modernization projects for over 30 FMCSA applications that result in a highly usable, responsive, flexible, secure, and interoperable ecosystem of services for the end users. A key component of this position institutionalizing a coherent methodology for solving engineering problems related to enterprise software at FMCSA."
    },
'''

In [ ]:
#Develop a function to preprocess the input text

def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()

    # Remove punctuation from the text
    text = re.sub('[^a-z]', ' ', text)

    # Remove numerical values from the text
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = ' '.join(text.split())

    return text


In [ ]:
# Apply to CV and JD
input_CV = preprocess_text(resume)
input_JD = preprocess_text(job_desc)

In [ ]:
#Now, here it comes the matching process.
#Utilizing the trained model, we get the document vectors for the resume and job description.
# Subsequently, we compute the cosine similarity between the two vectors to assess the correspondence between the resume and the job description.

# Model evaluation
model = Doc2Vec.load('cv_job_maching.model')
v1 = model.infer_vector(input_CV.split())
v2 = model.infer_vector(input_JD.split())
similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(round(similarity, 2))

73.99


In [ ]:
#Visualization and Notification

# Visualization
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = similarity,
    mode = "gauge+number",
    title = {'text': "Matching percentage (%)"},
    #delta = {'reference': 100},
    gauge = {
        'axis': {'range': [0, 100]},
        'steps' : [
            {'range': [0, 50], 'color': "#FFB6C1"},
            {'range': [50, 70], 'color': "#FFFFE0"},
            {'range': [70, 100], 'color': "#90EE90"}
        ],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 100}}))

fig.update_layout(width=600, height=400)  # Adjust the width and height as desired
fig.show()

# Print notification
if similarity < 50:
    print(colored("Low chance, need to modify your CV!", "red", attrs=["bold"]))
elif similarity >= 50 and similarity < 70:
    print(colored("Good chance but you can improve further!", "yellow", attrs=["bold"]))
else:
    print(colored("Excellent! You can submit your CV.", "green", attrs=["bold"]))

Excellent! You can submit your CV.
